In [1]:
import pandas as pd
import os

In [2]:
data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/raw_data/'
cleaned_data_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/cleaned_data/'
variables_path = '/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/variables/'

In [3]:
frims_rc_id_mapping = pd.read_csv('/home/krishna/IDS-DRR-Data-Pipeline/Sources/FRIMS/data/frims_rc_id_mapping.csv')
frims_rc_id_mapping = frims_rc_id_mapping[['object_id','RC_Id']]

In [17]:
df = pd.read_excel(data_path+'FRIMS_Inf_Damage_Data_21_july_2023.xlsx',
                   sheet_name=0)
df = df.rename(columns = {'Revenue Circle Id': 'RC_Id'})

In [10]:
# Updated dataset
df = pd.read_csv(data_path + 'Infrastructure Damage.csv')
df['Report_Date'] = pd.to_datetime(df['Report_Date'])
df.head()

,Report_Date,District_Id,District_Name,DFC_Id,RC_Id,Block_Name,Infrastructure_Damage_Type,Name,Location,Longitude,Latitude,Remark,Department
0,2021-05-15,2,Baksa,30,18.0,Jalah,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-15,15,Nalbari,14,145.0,Barkhetri,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-05-15,34,Chirang,105,45.0,Bengtal,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-05-15,15,Nalbari,140,144.0,Nalbari,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-05-15,2,Baksa,28,178.0,Barnagar RC part,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df = df.merge(frims_rc_id_mapping, on='RC_Id', how='left')

In [12]:
unmapped_rcs = set(df[df['object_id'].isnull()]['RC_Id'].dropna().to_list())
unmapped_rcs

set()

In [13]:
df['month'] = df['Report_Date'].dt.month
df['month'] = df.month.map("{:02}".format)
df['year'] = df['Report_Date'].dt.year

In [16]:
infra_damages_df = df.groupby(['year', 'month', 'object_id', 'Infrastructure_Damage_Type'])[['RC_Id']].count().reset_index()

In [17]:
infra_damages_df.columns = ['year', 'month', 'object_id', 'Type', 'num_damages']

In [18]:
infra_damages_df['year_month'] = infra_damages_df['year'].astype(str) + '_' + infra_damages_df['month'].astype(str)

In [19]:
variables = infra_damages_df.Type.unique()

In [22]:
for variable in variables:
    variable_df = infra_damages_df[infra_damages_df['Type'] == variable]
    
    for year_month in variable_df['year_month'].unique():
        variable_df_monthly = variable_df[variable_df['year_month']==year_month]  
        
        variable_df_monthly = variable_df_monthly.rename(columns = {'num_damages': variable})
        variable_df_monthly = variable_df_monthly[['object_id', variable]]
        if os.path.exists(variables_path+variable):
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)
        else:
            os.mkdir(variables_path+variable)
            variable_df_monthly.to_csv(variables_path+variable+'/{}_{}.csv'.format(variable, year_month), index=False)

In [ ]:
df[df['Revenue Circle Id']==61]

In [ ]:
frims_rc_id_mapping[frims_rc_id_mapping['RC_Id']==169]

In [23]:
variables

array(['Roads', 'Embankments affected', 'Bridge', 'Embankment breached'],
      dtype=object)